# Elon Musk Tweets Sentiment Analysis

I will be attempting to write an algorithmic trading bot which either longs or shorts TSLA stock depending on the sentiment analysis of Elon Musk's tweets. In order to see if such an algorithm would perform well, I will be gathering a [dataset](https://www.kaggle.com/ayhmrba/elon-musk-tweets-2010-2021) of his tweets from the years 2011-2021, and using them to backtest the strategy on QuantConnect. 


In [1]:
# from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')
# files.upload()

# !cp kaggle.json ~/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json

# !kaggle datasets download -d ayhmrba/elon-musk-tweets-2010-2021
# !unzip /content/elon-musk-tweets-2010-2021.zip

Mounted at /content/gdrive


In [ ]:
!pip install transformers

In [14]:
# import dependencies
import pandas as pd
import re
import csv

from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [4]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Kaggle/ElonMuskTweets/2021.csv")
df

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,urls,photos,video,thumbnail,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,0,1373819373090050048,1373669212271566858,1.616379e+12,2021-03-22 02:10:37,0,NaN,@bluemoondance74 @NASASpaceflight Going well. ...,en,[],[],44196397,44196397,elonmusk,Elon Musk,1,2,https://twitter.com/elonmusk/status/1373819373...,[],[],0,NaN,False,2678,234,134,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'bluemoondance74', 'name': 'R...",NaN,NaN,NaN,NaN
1,1,1373735946244431873,1373669212271566858,1.616359e+12,2021-03-21 20:39:07,0,NaN,@NASASpaceflight Hopefully will happen this de...,en,[],[],44196397,44196397,elonmusk,Elon Musk,7,20,https://twitter.com/elonmusk/status/1373735946...,[],[],0,NaN,False,7967,535,570,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'NASASpaceflight', 'name': 'C...",NaN,NaN,NaN,NaN
2,2,1373555480870621188,1373328330041229312,1.616316e+12,2021-03-21 08:42:00,0,NaN,@newscientist True,en,[],[],44196397,44196397,elonmusk,Elon Musk,7,8,https://twitter.com/elonmusk/status/1373555480...,[],[],0,NaN,False,15510,539,409,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'newscientist', 'name': 'New ...",NaN,NaN,NaN,NaN
3,3,1373507545315172357,1373263440391864323,1.616305e+12,2021-03-21 05:31:31,0,NaN,@cleantechnica I am accumulating resources to ...,en,[],[],44196397,44196397,elonmusk,Elon Musk,7,5,https://twitter.com/elonmusk/status/1373507545...,[],[],0,NaN,False,143476,10500,12483,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'cleantechnica', 'name': 'Cle...",NaN,NaN,NaN,NaN
4,4,1373492611231535111,1373357995288051718,1.616301e+12,2021-03-21 04:32:11,0,NaN,@CathieDWood When vast amounts of manufacturin...,en,[],[],44196397,44196397,elonmusk,Elon Musk,7,4,https://twitter.com/elonmusk/status/1373492611...,[],[],0,NaN,False,6568,376,425,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'CathieDWood', 'name': 'Cathi...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12334,12334,142881284019060736,142881284019060736,1.322901e+12,2011-12-03 08:22:07,0,NaN,That was a total non sequitur btw,en,[],[],44196397,44196397,elonmusk,Elon Musk,6,8,https://twitter.com/elonmusk/status/1428812840...,[],[],0,NaN,False,53,31,6,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
12335,12335,142880871391838208,142880871391838208,1.322900e+12,2011-12-03 08:20:28,0,NaN,"Great Voltaire quote, arguably better than Twa...",en,[],[],44196397,44196397,elonmusk,Elon Musk,6,8,https://twitter.com/elonmusk/status/1428808713...,[],[],0,NaN,False,29,7,25,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
12336,12336,142188458125963264,142188458125963264,1.322735e+12,2011-12-01 10:29:04,0,NaN,I made the volume on the Model S http://t.co/...,en,[],[],44196397,44196397,elonmusk,Elon Musk,4,10,https://twitter.com/elonmusk/status/1421884581...,['http://ow.ly/i/mtD7'],[],0,NaN,False,78,31,9,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
12337,12337,142179928203460608,142179928203460608,1.322733e+12,2011-12-01 09:55:11,0,NaN,Went to Iceland on Sat to ride bumper cars on ...,en,[],[],44196397,44196397,elonmusk,Elon Musk,4,9,https://twitter.com/elonmusk/status/1421799282...,[],['https://pbs.twimg.com/media/Afkf5DMCAAEJGFr....,1,https://pbs.twimg.com/media/Afkf5DMCAAEJGFr.jpg,False,189,32,15,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


The only columns we will need are the 'date' and 'tweet' columns. Also, we need to reverse them for the algorithm's sake. They need to be going from olders to latest, not vice versa.

In [5]:
df = df[['date', 'tweet']]
df = df[::-1].reset_index(drop=True)
df

,date,tweet
0,2010-06-04 18:31:57,"Please ignore prior tweets, as that was someon..."
1,2011-12-01 09:55:11,Went to Iceland on Sat to ride bumper cars on ...
2,2011-12-01 10:29:04,I made the volume on the Model S http://t.co/...
3,2011-12-03 08:20:28,"Great Voltaire quote, arguably better than Twa..."
4,2011-12-03 08:22:07,That was a total non sequitur btw
...,...,...
12334,2021-03-21 04:32:11,@CathieDWood When vast amounts of manufacturin...
12335,2021-03-21 05:31:31,@cleantechnica I am accumulating resources to ...
12336,2021-03-21 08:42:00,@newscientist True
12337,2021-03-21 20:39:07,@NASASpaceflight Hopefully will happen this de...


Much better. Lets look at a couple random tweets.

In [6]:
print(df['tweet'][115])
print(df['tweet'][742])
print(df['tweet'][1215])
print(df['tweet'][9211])
print(df['tweet'][11501])

The Model S beta endurance car just passed 150,000 miles on a single battery pack!
Will be doing the LA-NY family road trip over Spring Break. Made everyone watch National Lampoon's Vacation as prep.
@JeffBezos Not quite "rarest". SpaceX Grasshopper rocket did 6 suborbital flights 3 years ago &amp; is still around.  https://t.co/6j9ERKCNZl
@CNN @GavinNewsom  https://t.co/2KfHKLfUJh
@PhilippeCROIZON @SpaceX One day we will fly you on Starship


So, firstly we can see that a lot of Elon's tweets are either about random things or about his other companies, so we will need to filter them to make sure whether or not we should be using each tweet or not. 

Second, it is possible for Elon to share good news about Tesla, which has a probability of inflating TSLA stock price, thereby giving us a bit of alpha if we are quick enough. 

And thirdly, we can see that there are URLS in a lot of the tweets, which could be detremental to the sentiment analyzer, so we will have to remove them.

In [7]:
for i in range(len(df)):
    if "http" in df["tweet"][i]:
        urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|[?:%[0-9a-fA-F][0-9a-fA-F])+', df["tweet"][i])

        for url in urls:
            df["tweet"][i] = df["tweet"][i].replace(url, '{URL}')

In [8]:
print(df['tweet'][115])
print(df['tweet'][742])
print(df['tweet'][1215])
print(df['tweet'][9211])
print(df['tweet'][11501])

The Model S beta endurance car just passed 150,000 miles on a single battery pack!
Will be doing the LA-NY family road trip over Spring Break. Made everyone watch National Lampoon's Vacation as prep.
@JeffBezos Not quite "rarest". SpaceX Grasshopper rocket did 6 suborbital flights 3 years ago &amp; is still around.  {URL}
@CNN @GavinNewsom  {URL}
@PhilippeCROIZON @SpaceX One day we will fly you on Starship


Much better. I will save this new DF as a csv of its own to have for later.

In [9]:
df.to_csv("/content/gdrive/MyDrive/Colab Notebooks/Kaggle/ElonMuskTweets/ElonMuskTweetsPreProcessed.csv", index=False)

Now, since QuantConnect does not let us import the transformers library into its environement, we will have to perform the sentiment analysis on the data beforehand, and save it as a new csv which has scores instead of tweets. 

In [10]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

The way that the BERT sentiment analyzer works, is that first you pass the sentence/passage/tweet into the tokenizer which spits out a vector of tokenized words. Then, you pass the tokens into the model and it returns 5 scores, almost like 5 star reviews. The score with the highest number is the most likely one. Lets see an example

In [11]:
good_setence = "Wow that is amazing, I cannot believe it! Incredible! Fantastic!"
bad_sentence = "That was the most horrible thing I have ever experienced. Terrible! Never again!"

good_tokens = tokenizer.encode(good_setence, return_tensors="pt")
bad_tokens = tokenizer.encode(bad_sentence, return_tensors="pt")

print(good_tokens)
print(bad_tokens)

tensor([[  101, 94608, 10203, 10127, 39854,   117,   151, 25004, 22142, 10197,
           106, 81981,   106, 47088,   106,   102]])
tensor([[  101, 10203, 10140, 10103, 10889, 36129, 45795, 10301, 21973,   151,
         10574, 15765, 39183,   119, 50334,   106, 13362, 12590,   106,   102]])


In [12]:
good_results = model(good_tokens)
bad_results = model(bad_tokens)

print(good_results)
print(bad_results)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.6161, -2.4433, -1.5049,  0.4739,  4.2215]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[ 5.0263,  1.4021, -1.1469, -3.1746, -1.4358]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)


As we can see, the good sentence has a much higher number in the last logit which is equivalent to 5 stars, while the bad sentence has the highest number in the first logit spot, resembling a 1 star!

Now lets try it with a tweet from Elon. I'll use one of the tweets we printed earlier.

In [13]:
tweet = "The Model S beta endurance car just passed 150,000 miles on a single battery pack!"
tokens = tokenizer.encode(tweet, return_tensors="pt")
result = model(tokens)
print(result)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.5236, -0.9150, -0.6577,  0.3347,  1.3500]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)


Although the scores aren't as obvious as the good sentence example I provided, we can see that the analyzer is still able to pick up that this is in fact good news, and places the highest sentiment possibilty on the 5-star!

Now lets make a new CSV which has the additional column of 'score', which is the model's output for each tweet.

In [17]:

input_file = "/content/gdrive/MyDrive/Colab Notebooks/Kaggle/ElonMuskTweets/ElonMuskTweetsPreProcessed.csv"
output_file = "/content/gdrive/MyDrive/Colab Notebooks/Kaggle/ElonMuskTweets/ElonMuskTweetsFinal.csv"

with open(input_file, 'r', newline='') as f_in, open(output_file, 'w', newline='') as f_out:
    reader = csv.reader(f_in, delimiter=',')
    writer = csv.writer(f_out, delimiter=',')
    next(reader)
    writer.writerow(['time', 'tweet', 'score'])
    for line in reader:
        time = line[0]
        tweet = line[1]
        tokens = tokenizer.encode(tweet, return_tensors="pt")
        result = model(tokens)
        nums = result.split("([[")[1].split("]")[0].split(",")
        real_nums = [float(num) for num in nums]
        writer.writerow([time, tweet, real_nums])

In [51]:
new_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Kaggle/ElonMuskTweets/ElonMuskTweetsFinal.csv")
new_df

,date,tweet,score
0,2011-12-01 09:55:11,Went to Iceland on Sat to ride bumper cars on ...,"[-1.2518, -1.5825, -0.477, 0.9447, 1.9605]"
1,2011-12-01 10:29:04,I made the volume on the Model S {URL} go to ...,"[0.5312, 0.6482, 0.709, -0.5385, -1.2305]"
2,2011-12-03 08:20:28,"Great Voltaire quote, arguably better than Twa...","[-1.6414, -1.308, -0.092, 1.2261, 1.5263]"
3,2011-12-03 08:22:07,That was a total non sequitur btw,"[2.5593, 1.1319, 0.2132, -1.462, -1.8178]"
4,2011-12-04 03:33:52,Am reading a great biography of Ben Franklin b...,"[-2.5243, -2.5999, -0.979, 1.7844, 3.3992]"
...,...,...,...
12333,2021-03-21 04:32:11,@CathieDWood When vast amounts of manufacturin...,"[-2.3173, -1.2149, 0.2248, 1.753, 1.1714]"
12334,2021-03-21 05:31:31,@cleantechnica I am accumulating resources to ...,"[-2.0738, -1.7202, -0.2923, 1.505, 1.9235]"
12335,2021-03-21 08:42:00,@newscientist True,"[-0.1079, -0.8622, -0.1104, 0.1674, 0.7615]"
12336,2021-03-21 20:39:07,@NASASpaceflight Hopefully will happen this de...,"[-0.5384, -0.8527, -0.2464, 0.4502, 0.8817]"


There we go. Now we have a clean csv file which holds every tweet by Elon Musk from December 1st, 2011 till March 22nd, 2021. For each tweet we have the exact time it was tweeted, the contents of the tweet, and the sentiment analysis 5-number score generated by BERT. Now I can use this csv as a custom data input inside QuantConnect, and build and algorithm which acts upon TSLA stock every time there is a tweet involving words related to Tesla.